In [8]:
import pandas as pd
from collections import defaultdict, deque
import json
import numpy as np

In [9]:

ONTOLOGY_PATH = '/Users/Joe/Downloads/SNOMED_CF.csv'
Ontology_df = pd.read_csv(ONTOLOGY_PATH)

/Users/joe/Library/Python/3.8/lib/python/site-packages/IPython/core/interactiveshell.py:3251: DtypeWarning: Columns (24,28,30) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [14]:
Ontology_df#.head()

,Class ID,Preferred Label,Synonyms,Definitions,Obsolete,CUI,Semantic Types,Parents,AQ,ChildCount,...,SYNONYM MTH_SY,SYNONYM MTH_SYGB,SYNONYM OF,SYNONYM OP,SYNONYM PTGB,SYNONYM SY,SYNONYM SYGB,TUI,UMLS_CUI,id
0,http://ihtsdo.org/snomedct/clinicalFinding#284...,Physically threatening behavior,NaN,NaN,False,NaN,NaN,http://ihtsdo.org/snomedct/clinicalFinding#284...,NaN,NaN,...,NaN,NaN,NaN,NaN,Physically threatening behaviour,NaN,NaN,T033,C0562574,284615005
1,http://ihtsdo.org/snomedct/clinicalFinding#163...,O/E - nipple discharge - pus,NaN,NaN,False,NaN,NaN,http://ihtsdo.org/snomedct/clinicalFinding#163...,Clinical course~Courses~3106052025~263502005~2...,NaN,...,NaN,NaN,On examination - nipple discharge - pus (conte...,NaN,NaN,On examination - nipple discharge - pus,NaN,T033,C0437106,163463002
2,http://ihtsdo.org/snomedct/clinicalFinding#424...,Uninsured medical expenses,NaN,NaN,False,NaN,NaN,http://ihtsdo.org/snomedct/clinicalFinding#419...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,T033,C0237129,424553001
3,http://ihtsdo.org/snomedct/clinicalFinding#428...,Infection due to Brucella canis,NaN,NaN,False,NaN,NaN,http://ihtsdo.org/snomedct/clinicalFinding#757...,Episodicity~Episodicities~3258524025~246456000...,NaN,...,NaN,NaN,NaN,NaN,NaN,Brucella canis infection,NaN,T047,C0494040,428174001
4,http://ihtsdo.org/snomedct/clinicalFinding#404...,Alternating hemiplegia,NaN,NaN,False,NaN,NaN,http://ihtsdo.org/snomedct/clinicalFinding#505...,Severity~Severities~2507513025~246112005~27214...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,T047,C0278110,404689008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106616,http://ihtsdo.org/snomedct/clinicalFinding#785...,Abnormal number,NaN,NaN,False,NaN,NaN,http://ihtsdo.org/snomedct/clinicalFinding#107...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,T080,C0332594,78535000
106617,http://ihtsdo.org/snomedct/clinicalFinding#367...,PID with female sterility due to Chlamydia tra...,NaN,NaN,False,NaN,NaN,http://ihtsdo.org/snomedct/clinicalFinding#105...,Clinical course~Courses~3151874026~263502005~2...,NaN,...,NaN,NaN,PID with female sterility due to Chlamydia tra...,NaN,NaN,Pelvic inflammatory disease with female steril...,NaN,T047,C0276107,367504009
106618,http://ihtsdo.org/snomedct/clinicalFinding#369...,"Vaginal surgical margin, involved by tumor",NaN,NaN,False,NaN,NaN,http://ihtsdo.org/snomedct/clinicalFinding#370...,NaN,NaN,...,NaN,NaN,"Vaginal surgical margin, involved by tumor (qu...",NaN,"Vaginal surgical margin, involved by tumour",NaN,NaN,T034,C1298110,369647009
106619,http://ihtsdo.org/snomedct/clinicalFinding#219...,Injury undetermined whether accidentally or pu...,NaN,NaN,False,NaN,NaN,http://ihtsdo.org/snomedct/clinicalFinding#269...,Episodicity~Episodicities~2927447024~246456000...,NaN,...,NaN,NaN,Injury undetermined whether accidentally or pu...,NaN,NaN,NaN,NaN,T037,C0418554,219331002


In [11]:
# only keep the ID for each relation
Ontology_df['id'] = Ontology_df['Class ID'].apply(lambda x: x.split("#")[-1])

In [12]:
# build relations between parents (p) and children (c)
# both relation_c2p and relation_p2c are dicts of list of IDs becasue there can be multiple parents and children
relation_c2p = {}
c2name = {}
relation_p2c = defaultdict(set)
for i, row in Ontology_df.iterrows():
    current_id = row['id']
    parents = [p.split("#")[-1] for p in row['Parents'].split("|")]
    if current_id == "404684003": # root CF dont have parents
        parents = []
    if len(parents) == 0:
        print(current_id)
        
    c2name[current_id] = row["Preferred Label"].lower()
    relation_c2p[current_id] = parents
    for p in parents:
        relation_p2c[p].add(current_id)

404684003


In [13]:
# Root of clinical findings: id "404684003"
# You can find the id from https://bioportal.bioontology.org/ontologies/SNOMED_CF/?p=classes&conceptid=root
layer_1 = relation_p2c["404684003"] # find all children of root
relation_c2layer1 = defaultdict(set)
for c in c2name.keys():
    deq = deque()
    deq.append(c)
    while len(deq)>0:
        q = deq.popleft()
        if not q:
            relation_c2layer1[c] = set([])
            break
        if q == "Thing":
            print(c)
        tmp_list = relation_c2p[q]
        for t in tmp_list:
            if t in layer_1:
                relation_c2layer1[c].add(t)
            else:
                deq.append(t)

In [47]:
for key, value in relation_p2c.items():
    relation_p2c[key] = list(value)

for key, value in relation_c2layer1.items():
    relation_c2layer1[key] = list(value)

In [48]:
json.dump(dict(relation_c2layer1),
          open("/Users/Joe/Downloads/relation_c2layer1.json","w"),
          indent=4)
json.dump(c2name,
          open("/Users/Joe/Downloads/snomed_c2name.json","w"),
          indent=4)
json.dump(relation_p2c,
          open("/Users/Joe/Downloads/relation_p2c.json","w"),
          indent=4)

In [13]:
list(c2name.keys())[65753]

'441742003'

In [14]:
c2name['441742003']

'evaluation finding'

# find layer 2 relation for : "disease" and "finding by site", and "clinical history and observation", "nerological finding", 'evaluation finding'
'118234003',
'64572001',
'250171008',
'102957003'
'441742003'

In [127]:
c2layer1 = list(relation_c2layer1.values())
disease_mask = np.array([ 1 if '64572001' in relations else 0 for relations in c2layer1])
fingbysite_mask = np.array([ 1 if '118234003' in relations else 0 for relations in c2layer1])
observation_mask = np.array([ 1 if '250171008' in relations else 0 for relations in c2layer1])
nero_mask = np.array([ 1 if '102957003' in relations else 0 for relations in c2layer1])
eval_mask = np.array([ 1 if '441742003' in relations else 0 for relations in c2layer1])

In [92]:
def get_layer2_relation(concept_in_layer1, mask):
    layer_2 = relation_p2c[concept_in_layer1]
    relation_c2layer2 = defaultdict(set)
    for i, c in enumerate(c2name.keys()):
        if mask[i] == 0:
            relation_c2layer2[c] = set([])
            continue
        deq = deque()
        deq.append(c)
        while len(deq)>0:
            q = deq.popleft()
            if not q:
                relation_c2layer2[c] = set([])
                break
            tmp_list = relation_c2p[q]
            for t in tmp_list:
                if t in layer_2:
                    relation_c2layer2[c].add(t)
                else:
                    deq.append(t)
    for key, value in relation_c2layer2.items():
        relation_c2layer2[key] = list(value)
    print(len(relation_c2layer2))
    return relation_c2layer2

In [93]:
relation_c2layer2_disease = get_layer2_relation('64572001', disease_mask)
json.dump(dict(relation_c2layer2_disease),
          open("/data/zolab/joe/stressed_ensemble/data/relation_c2layer2_disease.json","w"),
          indent=4)

106621


In [100]:
relation_c2layer2_findingbysite = get_layer2_relation('118234003', fingbysite_mask)
json.dump(dict(relation_c2layer2_findingbysite),
          open("/data/zolab/joe/stressed_ensemble/data/relation_c2layer2_findingbysite.json","w"),
          indent=4)

106621


In [101]:
relation_c2layer2_observation = get_layer2_relation('250171008', observation_mask)
json.dump(dict(relation_c2layer2_observation),
          open("/data/zolab/joe/stressed_ensemble/data/relation_c2layer2_observation.json","w"),
          indent=4)

106621


In [102]:
relation_c2layer2_neuro = get_layer2_relation('102957003', nero_mask)
json.dump(dict(relation_c2layer2_neuro),
          open("/data/zolab/joe/stressed_ensemble/data/relation_c2layer2_neuro.json","w"),
          indent=4)

106621


In [128]:
relation_c2layer2_eval = get_layer2_relation('441742003', eval_mask)
json.dump(dict(relation_c2layer2_eval),
          open("/data/zolab/joe/stressed_ensemble/data/relation_c2layer2_eval.json","w"),
          indent=4)

106621


In [138]:
relation_p2c["441742003"]

['124983002',
 '442670009',
 '165825004',
 '301830001',
 '167975002',
 '168318001',
 '250435001',
 '275732003',
 '300544009',
 '310579000',
 '406117000',
 '125154007',
 '404527001',
 '37494006',
 '442666001',
 '365861007',
 '440661002',
 '314706002',
 '165816005',
 '406105002',
 '442635007',
 '166165005',
 '281276009',
 '250228007',
 '166432009',
 '166168007',
 '442618008',
 '165836006',
 '304294003',
 '251397002',
 '314526000',
 '15656008',
 '438508001',
 '365706007',
 '395028008',
 '365613002',
 '397918000',
 '168124002',
 '414878005',
 '167596001',
 '281261001',
 '66552009',
 '124989003',
 '166167002',
 '274683000',
 '312397004',
 '250371001',
 '395538009',
 '281279002',
 '365696009',
 '252097006',
 '118207001',
 '123830001',
 '314707006',
 '165668000',
 '167981005',
 '164970004',
 '168021008',
 '406010001',
 '124875007',
 '124873000',
 '72724002',
 '124990007',
 '274705002',
 '106146005',
 '70003006',
 '251406000',
 '365719006',
 '404558000',
 '62117008',
 '23506009',
 '250242004',